# KcBERT: Korean comments BERT


-깃헙 블로그 주소: https://github.com/Beomi/KcBERT

- KcBERT는 위와 같은 특성의 데이터셋에 적용하기 위해, 온라인 뉴스에서 댓글과 대댓글을 수집해, 토크나이저와 BERT모델을 처음부터 학습한 Pretrained BERT 모델입니다.
- KcBERT는 Huggingface의 Transformers 라이브러리를 통해 간편히 불러와 사용할 수 있습니다. (별도의 파일 다운로드가 필요하지 않습니다.)

In [36]:
!pip install transformers

In [37]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

In [38]:
#드라이브 연동
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
# # Torch GPU 설정
# device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device(device_type)

In [40]:
data1 = pd.read_excel("/content/drive/MyDrive/Prog_All/Data/ko_단발성_대화_데이터셋.xlsx", engine='openpyxl')
data2 = pd.read_excel("/content/drive/MyDrive/Prog_All/Data/ko_연속적_대화_데이터셋.xlsx", engine='openpyxl')

In [41]:
data1 = data1[['Sentence','Emotion']]
data2 = data2[['Unnamed: 1','Unnamed: 2']]
data2.drop([0],axis=0,inplace=True)
data2.rename(columns={'Unnamed: 1':'Sentence','Unnamed: 2':'Emotion'},inplace=True)
data2.replace('ㅍ','공포',inplace=True)
data2.replace(['분','분ㄴ'],'분노',inplace=True)
data2.replace(['ㅈ중립','중림','ㄴ중립','줄'],'분노',inplace=True)


<ipython-input-41-8cd63477b64b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.drop([0],axis=0,inplace=True)
<ipython-input-41-8cd63477b64b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.rename(columns={'Unnamed: 1':'Sentence','Unnamed: 2':'Emotion'},inplace=True)
<ipython-input-41-8cd63477b64b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.replace('ㅍ','공포',inplace=True)
<ipython-input-41-8cd63477b64b>:6: Sett

In [42]:
# nan 제거
data2 = data2.dropna(how='any')

In [43]:
data1['Emotion'].unique()
data2['Emotion'].unique()

array(['분노', '혐오', '중립', '놀람', '행복', '공포', '슬픔'], dtype=object)

In [44]:
data = pd.concat([data1,data2])
data.sample(n=10)

,Sentence,Emotion
16347,철판 두꺼운건 진짜 역대급이네 ㅉㅉ,분노
24102,박지성한테 털리고 손흥민한테 털리고 몇년후엔 이승우한테도 털릴려나,중립
28606,퇴진 100만 함성소리에 전율이 느껴졌다~~,행복
449,글타고 통증은 없는데34살인데 뿌그덕;;;;,공포
25542,자격증 응시료 10만원 1차 필기 2차 실기.,중립
2403,잘 다녀왔습니다!,중립
54495,"출발선부터 생각하기 까다로우면, 결승점에서부터 거꾸로 짚어 올라가는 편이 빠르지 않겠냐?",중립
34247,화 내세요? 난 언성 높이는 거 한번도 못봤는데~ 항상 웃으시잖아요,중립
49864,……나중에 하면 안 될까?,중립
850,집에 가는거야?,중립


In [45]:
data.drop_duplicates(['Sentence','Emotion'],inplace=True) # 중복행 제거
len(data)

90116

In [46]:
num_labeling_dics ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    '중립': 4,
    '행복': 5,
    '혐오': 6
  }

In [47]:
# 감정 -> 숫자 레이블링
for label_class in num_labeling_dics:
    data.loc[(data['Emotion'] == label_class), 'Emotion'] = num_labeling_dics[label_class]

In [48]:
# data_list = []
# for q, label in zip(data['Sentence'], data['Emotion'])  :
#     check_data = []
#     check_data.append(q)
#     check_data.append(str(label))
#     data_list.append(check_data)

In [49]:
texts = []
labels = []
for q, label in zip(data['Sentence'], data['Emotion'])  :
    texts.append(q)
    labels.append((label))

In [50]:
texts

['언니 동생으로 부르는게 맞는 일인가요..??',
 '그냥 내 느낌일뿐겠지?',
 '아직너무초기라서 그런거죠?',
 '유치원버스 사고 낫다던데',
 '근데 원래이런거맞나요',
 ' 남자친구가 떠날까봐요',
 '이거 했는데 허리가 아플수도 있나요? ;;',
 '내가불안해서꾸는걸까..',
 ' 일주일도 안 남았당...ㅠㅠ',
 '약은 최대한 안먹으려고 하는데좋은 음시있나요?0',
 ' 구직활동 하면서 남는시간은 뭘로 활용해야 되지..',
 '괜찮은분 같아서 괜히 조급해지네요 ㅜ',
 ' 제가 스타일을 바꾸면 더 어색하게만 변할것같아서 ㅠ0',
 '아내 있으면 여자 있는 술집가면 법 적으로 문제있나요?',
 ' 한소리들을지.... 도와주세요',
 '도와주실분 없나요..',
 '그냥 잡다한거말고학업에 관련된거 듣는게 더 좋은가요?',
 ' 가격은 반값....이거 질이 다른건가요??',
 '마지막에 말씀 드렸던대로 글은 이제 내릴게요의도치않게 실시간베스트글에 올라가서 남친이 볼거 같아서요 ㅠㅠ',
 '화장품회사다보니, 젊은 여자들이 많은거 같은데 걱정이네요..',
 ' 몸무게 1키로찌는건 아니겠죠?',
 'o자 다리 많이 심한가요?',
 ' 안하는게 나을까요??',
 '그정도로 더러운가요.........',
 ' 보통 가진통도 이렇게 오래가나요?',
 '여자가 술취해서 먼저 전화하는거 짜증나요???',
 '걱정이 큽니다',
 '이거 무슨 증상이죠??ㅜㅜ',
 ' 다름 아니라 제동생 걱정 입니다',
 '아무래도 무리겠죠?',
 '애인으로써 정말 걱정됩니다',
 '이러다 일주일전에 나오는거 아닌가몰라요',
 '경고먹을까 고민0',
 '극도로 예민하고 불안한 상태에요',
 ' 못받으면 어떡하죠',
 ' 안검하수도어울릴까요?',
 '근데 그게 사마귀 일수 있데요.',
 '이제 곧 내년이면 22살인 여대생입니다...요즘 고민이 있는데요...',
 ' 안하는게 좋은가요?',
 '자제할수있는 방법 뭐 없을까요??',
 '집에와서 또 운동하긴했는데지난 3주의 노력이 물거품이 될까봐

In [51]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [52]:
# print(data_list[0])
# print(data_list[6000])
# print(data_list[12000])
# print(data_list[18000])
# print(data_list[24000])
# print(data_list[30000])
# print(data_list[-1])

In [53]:
data['Emotion'].value_counts()

4    45641
1     9866
2     9238
3     7167
5     7015
6     5621
0     5568
Name: Emotion, dtype: int64

# WandB

In [54]:

!pip install wandb

In [55]:

!wandb login

wandb: Currently logged in as: tracy110410 (team_5g). Use `wandb login --relogin` to force relogin


In [56]:
import wandb
wandb.init(project="CJ_kcBERT", entity='tracy110410')


In [57]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}


In [58]:
def check_label(label):
  if label['공포'] == 0:
    return 0
  elif label['놀람'] == 1:
    return 1
  elif label['분노'] == 2:
    return 2
  elif label['슬픔'] == 3:
    return 3
  elif label['중립'] == 4:
    return 4
  elif label['행복'] == 5:
    return 5
  elif label['혐오'] == 6:
    return 6

In [59]:
# KcBERT 모델과 토크나이저 로드
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
# Setting parameters
max_length = 128
learning_rate = 1e-5
num_epochs = 5
batch_size = 64
random_seed=123

In [61]:
labels = torch.tensor(labels, dtype=torch.long)
dataset = CustomDataset(texts, labels, tokenizer, max_length)

In [62]:
# config
wandb.config ={
  "learning_rate": learning_rate,
  "epochs": num_epochs,
  "batch_size": batch_size,
  "seed": random_seed
}

In [63]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [64]:
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
train_data

In [66]:
# train_texts = []
# train_labels = []

# for idx, label in train_data.iterrows():
#   train_texts.append(label['Sentence'])
#   train_labels.append(check_label(label))


In [67]:
# test_texts = []
# test_labels = []
# for idx, label in test_data.iterrows():
#   test_texts.append(label['Sentence'])
#   test_labels.append(check_label(label))

In [68]:
data_train = CustomDataset(train_data, labels, tokenizer, max_length)
data_test = CustomDataset(test_data, labels, tokenizer, max_length)

In [69]:

# 옵티마이저 및 손실 함수 설정
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [70]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

model = model.to(device)

In [71]:

# 모델 재학습
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # 모델에 입력을 주어 예측을 생성합니다.
        outputs = model(input_ids, attention_mask=attention_mask)
        # 모델 출력에서 로짓(분류에 대한 점수)을 얻습니다.
        logits = outputs.logits
        # 손실을 계산합니다.
        loss = criterion(logits, labels)
        # 역전파를 통해 그래디언트 계산
        loss.backward()
        # 옵티마이저를 사용해 가중치를 업데이트
        optimizer.step()
        # 에포크 전체 손실을 누적합니다.
        total_loss += loss.item()

    # 에포크 평균 손실 계산
    avg_loss = total_loss / len(train_dataloader)
    # 에포크별 손실 출력
    print(f"Epoch {epoch+1}/{num_epochs} - Avg Loss: {avg_loss:.4f}")

    # 모델 평가
    model.eval()
    val_total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for val_batch in valid_dataloader:
            # Validation 데이터 가져오기
            val_input_ids = val_batch['input_ids']
            val_attention_mask = val_batch['attention_mask']
            val_labels = val_batch['label']

            val_input_ids = val_input_ids.to(device)
            val_attention_mask = val_attention_mask.to(device)
            val_labels = val_labels.to(device)

            # 모델 예측
            val_outputs = model(val_input_ids, attention_mask=val_attention_mask)
            val_logits = val_outputs.logits

            # 손실 계산
            val_loss = criterion(val_logits, val_labels)
            val_total_loss += val_loss.item()

            # 정확도 계산
            val_preds = val_logits.argmax(dim=1)
            correct += (val_preds == val_labels).sum().item()
            total += val_labels.size(0)

    val_avg_loss = val_total_loss / len(valid_dataloader)
    val_accuracy = correct / total
    print(f"Validation Loss: {val_avg_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/5 - Avg Loss: 0.9695
Validation Loss: 0.8684 - Validation Accuracy: 0.6890
Epoch 2/5 - Avg Loss: 0.7940
Validation Loss: 0.8460 - Validation Accuracy: 0.6914
Epoch 3/5 - Avg Loss: 0.6787
Validation Loss: 0.8828 - Validation Accuracy: 0.6891
Epoch 4/5 - Avg Loss: 0.5578
Validation Loss: 0.9350 - Validation Accuracy: 0.6804
Epoch 5/5 - Avg Loss: 0.4289
Validation Loss: 1.0674 - Validation Accuracy: 0.6633
